In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import uuid
import shutil
import os
from pathlib import Path

import numpy as np
import pandas as pd

from plaid.evaluation._structure_metrics import calculate_rmsd
from plaid.evaluation._tmalign import run_tmalign
from plaid.evaluation._perplexity import RITAPerplexity

from plaid.utils._misc import (
    extract_avg_b_factor_per_residue,
    parse_sequence_from_structure,
)
from plaid.utils._protein_properties import calculate_df_protein_property_mp

In [9]:
sample_dir = Path("/data/lux70/plaid/artifacts/samples/5j007z42/val_dist/f989_o1326_l144_s3")

In [12]:
import warnings

warnings.filterwarnings("ignore")

# Gather paths; sort should guarantee that samples are in the same order
generated_pdb_paths = glob.glob(str(sample_dir / "generated/structures/*pdb"))
inverse_generated_pdb_paths = glob.glob(
    str(sample_dir / "inverse_generated/structures/*pdb")
)

generated_pdb_paths.sort()
inverse_generated_pdb_paths.sort()
assert (
    len(generated_pdb_paths) == len(inverse_generated_pdb_paths)
)

# maybe run self-consistency (if phantom generated structures were generated)
phantom_generated_pdb_paths = glob.glob(
    str(sample_dir / "phantom_generated/structures/*pdb")
)
run_self_consistency = len(phantom_generated_pdb_paths) > 0

if run_self_consistency:
    phantom_generated_pdb_paths.sort()
    assert (
        len(generated_pdb_paths) == len(phantom_generated_pdb_paths)
    )


# Initialize dataframe
d = {
    "pdb_paths": [],
    "sequences": [],
    "inverse_generated_pdb_paths": [],
}



if run_self_consistency:
    d["phantom_generated_pdb_paths"] = []

# parse sequence directly from structure to make sure there are no mismatches
print("Parsing sequences from structures")
for i, p in enumerate(generated_pdb_paths):
    d["pdb_paths"].append(p)
    d["inverse_generated_pdb_paths"].append(inverse_generated_pdb_paths[i])

    if run_self_consistency:
        d["phantom_generated_pdb_paths"].append(phantom_generated_pdb_paths[i])

    with open(p, "r") as f:
        pdbstr = f.read()

    sequence = parse_sequence_from_structure(pdbstr)
    d["sequences"].append(sequence)


Parsing sequences from structures


In [15]:
df = pd.DataFrame(d)
df.head()

print("Calculating average pLDDT")
df["plddt"] = df.apply(
    lambda row: np.mean(extract_avg_b_factor_per_residue(row["pdb_paths"])),
    axis=1,
)

print("Calculating ccRMSD")
df["ccrmsd"] = df.apply(
    lambda row: calculate_rmsd(
        row["pdb_paths"], row["inverse_generated_pdb_paths"]
    ),
    axis=1,
)

print("Calculating cctm")
df["cctm"] = df.apply(
    lambda row: run_tmalign(
        row["pdb_paths"], row["inverse_generated_pdb_paths"]
    ),
    axis=1,
)

df["designable"] = df.ccrmsd < 2

# run self-consistency metrics, if applicable:
if run_self_consistency:
    print("Calculating scRMSD")
    df["scrmsd"] = df.apply(
        lambda row: calculate_rmsd(
            row["pdb_paths"], row["phantom_generated_pdb_paths"]
        ),
        axis=1,
    )

    print("Calculating sctm")
    df["sctm"] = df.apply(
        lambda row: run_tmalign(
            row["pdb_paths"], row["phantom_generated_pdb_paths"]
        ),
        axis=1,
    )


Calculating average pLDDT
Calculating ccRMSD
Calculating cctm
Calculating scRMSD
Calculating sctm


In [16]:
from plaid.utils import pdb_path_to_biotite_atom_array
from biotite.application.dssp import DsspApp

structure_atom_arrays = [pdb_path_to_biotite_atom_array(p) for p in df.pdb_paths]

In [21]:
import os
import shutil
from pathlib import Path
from typing import Optional


In [23]:
DSSP_PATH = get_mkdssp_path()

In [27]:
out = DsspApp.annotate_sse(structure_atom_arrays[0], DSSP_PATH)

In [29]:
dssp_annotation = "".join(out)

In [30]:
dssp_annotation

'CCCCCCCCCCCCCCCTTTGGGGSCCSSCCCCCCCCCCCCSSCCCCCCCCCCSSCCGGGSTTTTTTHHHHHHGGGCCCHHHHHHHHHHHHHHHHGGGSCSCCCCCHHHHHHHHHHHCGGGGCCCCCCCCCCCSCSCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCSCCCCCTTTTSCCCCCCCCCCSCGGGSCSCCCSSCCCCSCCCTTSCSSCCCCCCCCCTTHHHHHTTCCCCCHHHHHHHHHTTTCCCCC'

In [34]:
alpha_percentage = np.array([x == "C" for x in dssp_annotation]).sum() / len(dssp_annotation)
beta_percentage = np.array([x == "E" for x in dssp_annotation]).sum() / len(dssp_annotation)

In [35]:
print(coil_percentage, alpha_percentage)

0.6111111111111112 0.6111111111111112


In [ ]:
beta_percentage = [c == "E" metrics["sample_pct_beta"] = mean([c == "E" for c in dssp_sample])
metrics["sample_pct_alpha"] = mean([c == "H" for c in dssp_sample])